<a href="https://colab.research.google.com/github/WolfgangNS/GTF-to-matrix/blob/main/Renn_Lab_TPM_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#courtesy of ChatGPT

#data pipeline:
#https://chatgpt.com/c/4c7769c8-c3a3-4562-b962-9ff1357a9bdd

#data download:
#https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5769613
#GSM5769613_SR02_merged.gtf.gz

#-----
#upload .gtf file to /content directory

### Download mappings (BioMart)

In [4]:
#made into Python variable so the query is easier to read :)
#https://useast.ensembl.org/biomart/martview

import pandas as pd
import urllib.parse

query = (
    '<?xml version="1.0" encoding="UTF-8"?>'
    '<!DOCTYPE Query>'
    '<Query  virtualSchemaName = "default" formatter = "TSV" header = "0" uniqueRows = "0" count = "" datasetConfigVersion = "0.6" >'
    '<Dataset name = "hburtoni_gene_ensembl" interface = "default" >'
    '<Attribute name = "hburtoni_paralog_ensembl_gene" />'
    '<Attribute name = "oniloticus_homolog_ensembl_gene" />'
    '</Dataset>'
    '</Query>'
)
encoded_query = urllib.parse.quote(query)
wget_command = f"""
wget -q --show-progress -O genes.tsv http://www.ensembl.org/biomart/martservice?query={encoded_query}
"""

print(wget_command)
!{wget_command}

orthologs = pd.read_csv('genes.tsv',
                        sep='\t',
                        names=['hburtoni_paralog_ensembl_gene',
                                'oniloticus_homolog_ensembl_gene'])
orthologs.head(10)


wget -q --show-progress -O genes.tsv http://www.ensembl.org/biomart/martservice?query=%3C%3Fxml%20version%3D%221.0%22%20encoding%3D%22UTF-8%22%3F%3E%3C%21DOCTYPE%20Query%3E%3CQuery%20%20virtualSchemaName%20%3D%20%22default%22%20formatter%20%3D%20%22TSV%22%20header%20%3D%20%220%22%20uniqueRows%20%3D%20%220%22%20count%20%3D%20%22%22%20datasetConfigVersion%20%3D%20%220.6%22%20%3E%3CDataset%20name%20%3D%20%22hburtoni_gene_ensembl%22%20interface%20%3D%20%22default%22%20%3E%3CAttribute%20name%20%3D%20%22hburtoni_paralog_ensembl_gene%22%20/%3E%3CAttribute%20name%20%3D%20%22oniloticus_homolog_ensembl_gene%22%20/%3E%3C/Dataset%3E%3C/Query%3E

genes.tsv               [      <=>           ]  13.22M   913KB/s    in 17s     


,hburtoni_paralog_ensembl_gene,oniloticus_homolog_ensembl_gene
0,NaN,NaN
1,ENSHBUG00000023832,ENSONIG00000040832
2,ENSHBUG00000019623,ENSONIG00000040832
3,ENSHBUG00000016228,ENSONIG00000040832
4,ENSHBUG00000005138,ENSONIG00000040832
5,ENSHBUG00000016083,ENSONIG00000040832
6,ENSHBUG00000019072,ENSONIG00000040832
7,ENSHBUG00000008100,ENSONIG00000040832
8,ENSHBUG00000013528,ENSONIG00000040832
9,ENSHBUG00000012627,ENSONIG00000040832


In [5]:
#Again, but different
query = (
    '<?xml version="1.0" encoding="UTF-8"?>'
    '<!DOCTYPE Query>'
    '<Query  virtualSchemaName = "default" formatter = "TSV" header = "0" uniqueRows = "0" count = "" datasetConfigVersion = "0.6" >'
    '<Dataset name = "hburtoni_gene_ensembl" interface = "default" >'
    '<Attribute name = "ensembl_gene_id" />'
    '<Attribute name = "entrezgene_accession" />'
    '</Dataset>'
    '</Query>'
)
encoded_query = urllib.parse.quote(query)
wget_command = f"""
wget -q --show-progress -O genes2.tsv http://www.ensembl.org/biomart/martservice?query={encoded_query}
"""

print(wget_command)
!{wget_command}

ens_to_enz = pd.read_csv('genes2.tsv',
                        sep='\t',
                        names=['ensembl_gene_id',
                                'entrezgene_accession'])
ens_to_enz.head(10)


wget -q --show-progress -O genes2.tsv http://www.ensembl.org/biomart/martservice?query=%3C%3Fxml%20version%3D%221.0%22%20encoding%3D%22UTF-8%22%3F%3E%3C%21DOCTYPE%20Query%3E%3CQuery%20%20virtualSchemaName%20%3D%20%22default%22%20formatter%20%3D%20%22TSV%22%20header%20%3D%20%220%22%20uniqueRows%20%3D%20%220%22%20count%20%3D%20%22%22%20datasetConfigVersion%20%3D%20%220.6%22%20%3E%3CDataset%20name%20%3D%20%22hburtoni_gene_ensembl%22%20interface%20%3D%20%22default%22%20%3E%3CAttribute%20name%20%3D%20%22ensembl_gene_id%22%20/%3E%3CAttribute%20name%20%3D%20%22entrezgene_accession%22%20/%3E%3C/Dataset%3E%3C/Query%3E

genes2.tsv              [                <=> ] 622.62K  99.5KB/s    in 6.3s    


,ensembl_gene_id,entrezgene_accession
0,ENSHBUG00000023839,NaN
1,ENSHBUG00000000727,LOC102288583
2,ENSHBUG00000023838,hps1
3,ENSHBUG00000023837,LOC102292000
4,ENSHBUG00000023836,LOC102301706
5,ENSHBUG00000023835,slc25a29
6,ENSHBUG00000023834,LOC102298492
7,ENSHBUG00000023833,agap2
8,ENSHBUG00000023832,LOC102291978
9,ENSHBUG00000023831,LOC106633135


In [6]:
joined_df = pd.merge(ens_to_enz, orthologs, left_on='ensembl_gene_id', right_on='hburtoni_paralog_ensembl_gene', how='inner')

joined_df = joined_df.dropna()
joined_df = joined_df.drop(columns=['hburtoni_paralog_ensembl_gene'])

before_rows = len(ens_to_enz)
after_rows = len(joined_df)

print("Number of rows before joining:", before_rows)
print("Number of rows after joining and removing NaNs:", after_rows)
print("Preview of the resulting DataFrame:")
print(joined_df.head())

Number of rows before joining: 23922
Number of rows after joining and removing NaNs: 261369
Preview of the resulting DataFrame:
      ensembl_gene_id entrezgene_accession oniloticus_homolog_ensembl_gene
0  ENSHBUG00000000727         LOC102288583              ENSONIG00000040832
1  ENSHBUG00000000727         LOC102288583              ENSONIG00000035632
2  ENSHBUG00000000727         LOC102288583              ENSONIG00000008077
3  ENSHBUG00000000727         LOC102288583              ENSONIG00000041447
4  ENSHBUG00000000727         LOC102288583              ENSONIG00000013172


In [7]:
joined_df.to_csv("/content/entrez-ensembl-mappings.tsv", sep='\t')

### Download .tar (.gtf files inside)

In [ ]:
#https://ftp.ncbi.nlm.nih.gov/geo/series/GSE192nnn/GSE192958/suppl/
!wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE192nnn/GSE192958/suppl/GSE192958_RAW.tar
!tar -xvf GSE192958_RAW.tar -k

### Create gene count matrix from samples (HTSeq)

In [ ]:
!pip install HTSeq scanpy

In [ ]:
#15m for code below

In [9]:
import os
import HTSeq
import numpy as np
import pandas as pd

# Directory containing .gtf files
directory = "/content"

# Function to extract gene expression counts from a single .gtf file
def extract_counts(gtf_file):
    counts = {}
    features = HTSeq.GFF_Reader(gtf_file)
    for feature in features:
        if feature.type == "transcript":
            gene_name = feature.attr.get("gene_name")
            ref_gene_name = feature.attr.get("ref_gene_name")
            tpm = feature.attr.get("TPM")  # Get TPM attribute
            if tpm is not None:
                tpm_value = float(tpm)
                if gene_name is not None:
                    counts[gene_name] = counts.get(gene_name, 0) + tpm_value
                elif ref_gene_name is not None:
                    counts[ref_gene_name] = counts.get(ref_gene_name, 0) + tpm_value
    return counts

# Iterate over .gtf files in the directory
all_counts = {}
for filename in os.listdir(directory):
    if filename.endswith(".gtf"):
        filepath = os.path.join(directory, filename)
        sample_name = os.path.splitext(filename)[0]
        counts = extract_counts(filepath)
        all_counts[sample_name] = counts

# Convert counts to DataFrame and save as TSV
gene_names = sorted(set().union(*(counts.keys() for counts in all_counts.values())))

sample_names = list(all_counts.keys())
count_matrix = np.zeros((len(sample_names), len(gene_names)), dtype=int)
for i, sample_name in enumerate(sample_names):
    counts = all_counts[sample_name]
    for j, gene_name in enumerate(gene_names):
        count_matrix[i, j] = counts.get(gene_name, 0)

count_df = pd.DataFrame(count_matrix, index=sample_names, columns=gene_names)
count_df.to_csv("/content/output.tsv", sep='\t')

In [10]:
count_df.head()

,LOC102288483,LOC102288484,LOC102288486,LOC102288487,LOC102288488,LOC102288489,LOC102288492,LOC102288493,LOC102288495,LOC102288499,...,zswim5,zswim6,zswim7,zswim8,zw10,zwilch,zyg11b,zyx,zzef1,zzz3
GSM5769620_SR11_merged,15,0,0,0,0,0,1,0,0,1,...,44,1,14,16,2,0,3,1,13,14
GSM5769626_SR17_merged,1,0,2,0,0,0,1,0,0,1,...,25,29,11,26,2,0,8,0,14,50
GSM5769630_SR21_merged,1,0,1,0,0,0,0,0,0,2,...,73,1,12,13,1,0,1,1,13,11
GSM5769655_SR47_merged,1,0,0,0,0,0,1,0,0,0,...,27,33,8,28,3,0,11,0,15,41
GSM5769647_SR39_merged,2,0,0,0,0,1,1,0,0,2,...,66,9,6,27,2,0,17,0,17,39


In [11]:
!head output.tsv

	LOC102288483	LOC102288484	LOC102288486	LOC102288487	LOC102288488	LOC102288489	LOC102288492	LOC102288493	LOC102288495	LOC102288499	LOC102288501	LOC102288506	LOC102288507	LOC102288513	LOC102288514	LOC102288515	LOC102288516	LOC102288518	LOC102288520	LOC102288525	LOC102288526	LOC102288527	LOC102288528	LOC102288530	LOC102288531	LOC102288533	LOC102288534	LOC102288538	LOC102288541	LOC102288544	LOC102288545	LOC102288546	LOC102288547	LOC102288550	LOC102288551	LOC102288553	LOC102288554	LOC102288555	LOC102288559	LOC102288560	LOC102288561	LOC102288562	LOC102288563	LOC102288566	LOC102288569	LOC102288570	LOC102288572	LOC102288575	LOC102288577	LOC102288578	LOC102288581	LOC102288582	LOC102288583	LOC102288586	LOC102288587	LOC102288590	LOC102288592	LOC102288594	LOC102288595	LOC102288597	LOC102288598	LOC102288601	LOC102288602	LOC102288604	LOC102288605	LOC102288606	LOC102288608	LOC102288611	LOC102288614	LOC102288615	LOC102288618	LOC102288628	LOC102288630	LOC102288634	LOC102288635	LOC102288637	LOC10228863

In [12]:
##Hash table lookup

gene_mapping = dict(zip(joined_df["entrezgene_accession"], joined_df["oniloticus_homolog_ensembl_gene"]))
# count_df = pd.read_csv("/content/output.tsv", sep='\t', index_col=0)
count_df.columns = [gene_mapping.get(col, col) for col in count_df.columns]
transposed_count_df = count_df.transpose()
transposed_count_df.to_csv("/content/transposed_output.tsv", sep='\t')

In [13]:
transposed_count_df.sort_values(by=['GSM5769615_SR05_merged'], ascending=False).head()

,GSM5769620_SR11_merged,GSM5769626_SR17_merged,GSM5769630_SR21_merged,GSM5769655_SR47_merged,GSM5769647_SR39_merged,GSM5769650_SR42_merged,GSM5769644_SR35_merged,GSM5769651_SR43_merged,GSM5769648_SR40_merged,GSM5769642_SR33_merged,...,GSM5769637_SR28_merged,GSM5769635_SR26_merged,GSM5769616_SR06_merged,GSM5769614_SR03_merged,GSM5769639_SR30_merged,GSM5769623_SR14_merged,GSM5769652_SR44_merged,GSM5769649_SR41_merged,GSM5769638_SR29_merged,GSM5769622_SR13_merged
ENSONIG00000018330,10032,8530,12212,8252,10643,11292,6390,10134,10063,4345,...,11726,6699,7132,9639,6095,7616,7934,8617,16420,8304
ENSONIG00000040079,4579,3910,4857,3924,5367,3579,2733,2703,3121,4870,...,4798,4048,4037,4514,5665,4412,3117,3489,4375,5842
ENSONIG00000038569,3353,3072,4705,3086,4947,3266,2321,2409,2729,4355,...,4610,3577,3772,4062,4594,5018,2581,3253,3492,6413
ENSONIG00000009481,7849,5488,7824,5715,4856,6797,6766,6016,4592,5906,...,7273,6492,6835,7217,6022,5759,5242,5537,7053,7914
LOC102310417,4095,4699,6525,3988,3076,4885,4615,4953,5014,3997,...,5282,4130,4948,5285,4832,4241,4244,4921,8044,3616


### Phenotype data

In [ ]:
!wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE192nnn/GSE192958/miniml/GSE192958_family.xml.tgz
!tar -zxvf GSE192958_family.xml.tgz

In [40]:
import xml.etree.ElementTree as ET
tree = ET.parse('GSE192958_family.xml')
root = tree.getroot()
with open('attributes.tsv', 'w') as tsvfile:
    tsvfile.write('GSM_id\tsamplename\n')
    for sample in root.findall('{http://www.ncbi.nlm.nih.gov/geo/info/MINiML}Sample'):
        sample_id = sample.attrib['iid']
        for subsample in sample.findall('{http://www.ncbi.nlm.nih.gov/geo/info/MINiML}Title'):
            # print(subsample.text)
            # print(sample.attrib['iid'])
            GSM_id = subsample.text
            samplename = sample.attrib['iid']
            tsvfile.write(f'{GSM_id}\t{samplename}\n')


In [43]:
!head -n 10 attributes.tsv

GSM_id	samplename
R02_WS_02	GSM5769613
R02_WS_03	GSM5769614
B02_LS_05	GSM5769615
B14_WS_06	GSM5769616
B14_LS_07	GSM5769617
R02_WS_08	GSM5769618
B02_WS_09	GSM5769619
B02_WS_11	GSM5769620
B14_LS_12	GSM5769621


### (Miscellaneous)

In [ ]:
#Briliant solution: https://unix.stackexchange.com/a/22624
!tail -n 420 GSE192958_series_matrix.txt \
  | head -30